# Tract patch for LSST stuff

To start, let's look at some realistic tract patch from LSDB catalogs. This is just one of the commissioning datasets, and I'll just show the coordinates and tract patch, since that's pretty non-controversial.

In [1]:
import lsdb

In [2]:
catalog = lsdb.read_hats("/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_14/dia_object_forced_source")

single_frame = catalog.partitions[0].compute()

In [3]:
single_frame[["coord_ra", "coord_dec", "tract", "patch"]].head(10)

,coord_ra,coord_dec,tract,patch
_healpix_29,,,,
9194470118511706,38.119649,5.96977,10464,9
9194470480217710,38.11887,5.97114,10464,9
9194500429915293,38.120496,5.979788,10464,9
9194506361451775,38.108313,5.981478,10464,9
9194527184754609,38.117899,5.993882,10464,9
9194527184754609,38.117899,5.993882,10464,9
9194652578793572,38.099146,6.00414,10464,9
9194652578793572,38.099146,6.00414,10464,9
9195005870293349,38.054808,6.009057,10463,0


So I'm pretty sure that 

```
tract = 10464
patch = 9
```

is going to have radec values around (vague estimates here):

```
ra = 38.1 +/- .02
dec = 5.96 +/- .02
```

Let's get some data from butler's skymap, and see if that's true!!

In [4]:
import lsst.daf.butler as dafButler

In [5]:
config = "/repo/dp1"
butler = dafButler.Butler(config, collections="LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260")
skymap = butler.get("skyMap")
skymap

In [6]:
skymap.getRaDecRange(10464)

(Angle(38.091286307053942, degrees),
 Angle(39.585062240663902, degrees),
 Angle(5.9504132231404938, degrees),
 Angle(7.4380165289256199, degrees))

NOT TOO BAD.

In [7]:
tract_info = skymap.generateTract(10464)
tract_info.getPatchInfo(9).outer_sky_polygon

ConvexPolygon([UnitVector3d(0.7840209092505115, 0.6123114996987662, 0.10191094737391646), UnitVector3d(0.7819876346738198, 0.6149053601306285, 0.10191534379015939), UnitVector3d(0.7817220686637413, 0.614690914494313, 0.10519356920565955), UnitVector3d(0.7837554409506494, 0.6120969502793704, 0.10518903098207144)])

So maybe we can take just the skymap config, and reconstruct everything we need?

In [8]:
from lsst.skymap.ringsSkyMap import RingsSkyMap

my_map = RingsSkyMap(skymap.config)
my_map

In [9]:
my_map.getRaDecRange(10464)

(Angle(38.091286307053942, degrees),
 Angle(39.585062240663902, degrees),
 Angle(5.9504132231404938, degrees),
 Angle(7.4380165289256199, degrees))

NOT TOO BAD EITHER!!